In [3]:
import matplotlib.pyplot as plt

In [4]:
import plotly.graph_objects as go
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Crear funciones para extraer rango de fechas, realizar el preprocesamiento y graficar

In [5]:
def rango_fechas(df, columna_fecha):
    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    df_2 = df.copy()
    df_2['fecha'] = df_2[columna_fecha].dt.date
    fecha_inicial = df_2['fecha'].min()
    fecha_final = df_2['fecha'].max()
    print('Fecha Inicial: ', fecha_inicial); print('Fecha Final: ', fecha_final)

In [6]:
def preprocesamiento(df, columna_fecha, columna_id_usuario):
    df_temp = df[[columna_fecha, columna_id_usuario]]
    df_temp[columna_fecha] = pd.to_datetime(df_temp[columna_fecha])
    df_temp['fecha_creacion'] = df_temp[columna_fecha].dt.date
    df_temp.drop(columna_fecha, axis=1, inplace=True)
    df_temp.sort_values('fecha_creacion', ascending=True, inplace=True)
    df_group = df_temp.groupby('fecha_creacion')[columna_id_usuario].count().reset_index()
    df_group['fecha_creacion'] = pd.to_datetime(df_group['fecha_creacion'])
    return df_group
    

In [7]:
def grafico_fechas(df, columna_fecha, columna_id_usuario, fecha_inicio, fecha_fin):
    
    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    df_rango = df.copy()
    df_rango.set_index(columna_fecha, inplace=True)
    
    
    # Selecciona el rango de fechas
    df_rango = df_rango.loc[fecha_inicio:fecha_fin]

    # Calcula el valor máximo, mínimo y medio de la serie
    maximo = df_rango[columna_id_usuario].max()
    minimo = df_rango[columna_id_usuario].min()
    medio = df_rango[columna_id_usuario].mean()

    # Crea el gráfico
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_rango.index, y=df_rango[columna_id_usuario], mode='lines+markers', name='num_id'))

    fig.add_trace(
        go.Scatter(
            x=[df_rango.index.min(), df_rango.index.max()], 
            y=[maximo, maximo], 
            mode='lines', 
            line=dict(color='Red', width=1, dash="dashdot"),
            name=f'Máximo: {maximo}'
        )
    )

    fig.add_trace(
        go.Scatter(
            x=[df_rango.index.min(), df_rango.index.max()], 
            y=[minimo, minimo], 
            mode='lines', 
            line=dict(color='Green', width=1, dash="dashdot"),
            name=f'Mínimo: {minimo}'
        )
    )

    fig.add_trace(
        go.Scatter(
            x=[df_rango.index.min(), df_rango.index.max()], 
            y=[medio, medio], 
            mode='lines', 
            line=dict(color='Blue', width=1, dash="dashdot"),
            name=f'Medio: {medio:.2f}'  # Formateado a 2 decimales
        )
    )

    fig.update_layout(autosize = False, width = 1700, height = 600)

    fig.show()

## BD 1: Full Extranet Users

In [8]:
path = '../Datos/full_extranet.csv'

In [9]:
full = pd.read_csv(path)

In [10]:
full.head(1).transpose()

,0
CodigoID,80896
id_cli,78418
FBAFirstName,RAFAEL AMERICO
FBALastName,MORETA CASTILLO
TipoIdentificacion,2.0
TipoIdentificacionNombre,Cédula
ProfileARS,CPE
ProfileRG,PER
ProfileName,Cliente Personal
TELEFONO,8295485672


In [11]:
full.ProfileARS.value_counts()

ProfileARS
CPE     72201
CCO       312
PRE       220
INT       147
PSU        16
ADOM        1
CMA         1
AERO        1
BAN         1
LJ          1
Name: count, dtype: int64

In [12]:
full.MobileAlias.value_counts()

MobileAlias
                                    72162
jfernandez@swisherdominicana.com        1
Name: count, dtype: int64

In [13]:
full.Perfil.value_counts()

Perfil
Cliente Personal       72200
Cliente Corporativo      280
Prestadores               14
Intermediario              6
Aerodom                    1
Casa del Conductor         1
Aeroambulancia             1
LOJACK                     1
Name: count, dtype: int64

In [14]:
full.MobileAlias.value_counts()

MobileAlias
                                    72162
jfernandez@swisherdominicana.com        1
Name: count, dtype: int64

In [15]:
full.IsAnonymous.value_counts()

IsAnonymous
False    72162
Name: count, dtype: int64

In [16]:
rango_fechas(full, 'Created')

Fecha Inicial:  2013-05-15
Fecha Final:  2023-06-05


In [17]:
full_group = preprocesamiento(df=full, columna_fecha='Created', columna_id_usuario='numero_identificacion')

In [18]:
grafico_fechas(
    df=full_group,
    columna_fecha='fecha_creacion',
    columna_id_usuario='numero_identificacion',
    fecha_inicio='2015-01-01',
    fecha_fin='2023-06-05'
)

## BD 2: Historico FBA

In [19]:
path = '../Datos/historico_app.csv'
historico = pd.read_csv(path)

In [20]:
historico

,Id_cli,Perfil,tipo_id,num_id,IdentificacionPerfil,FBAFirstName,FBALastName,FBAUser,FBAComp,TELEFONO,Creado
0,NaN,Cliente Personal,D,00118941673,N987374,ANTONIO JOSE,PINEDA RAMOS,antonio_pineda26@hotmail.com,Ambos,8.093507e+09,2014-03-05
1,NaN,Cliente Personal,D,01200026886,N232927,Neyda Estela,Pérez Turbí,neyda_turbi@hotmail.com,Ambos,8.294234e+09,2014-03-24
2,NaN,Cliente Personal,D,00114081433,N239820,DOLORES MARGARITA,POZO PERELLO,dpozo@universal.com.do,Ambos,8.292928e+09,2014-04-17
3,NaN,Cliente Personal,D,04900135692,NaN,RICARDO ANTONIO,SANTOS RONDON,Multirimart@hotmail.com,SEGUROS,NaN,2014-04-17
4,NaN,Cliente Personal,D,00112968607,N1151939,YENY LUZ,GONZALEZ CORNIEL,Jenygonzalezz@hotmail.com,Ambos,NaN,2014-04-17
...,...,...,...,...,...,...,...,...,...,...,...
61067,NaN,Cliente Personal,D,40226234165,N2197791,ESTHER ELIZABETH,FELIZ ESPINAL,estherfelizespinal@outlook.com,Ambos,8.498694e+09,2022-04-05
61068,NaN,Cliente Personal,D,40225351804,N1677815,REYNA ALTARACIA,MUNOZ GRULLON,reynaaltagracia176@gmail.com,Ambos,8.296060e+09,2022-04-05
61069,NaN,Cliente Personal,D,40224570305,N1824151,MANUEL ALEJANDRO,MONEGRO MERCEDES,manuelmonegro0@gmail.com,Ambos,8.295464e+09,2022-04-05
61070,NaN,Cliente Personal,D,40220074872,N2081075,IRENO ALGEDIS,DIAZ FIRPO,algedi.01@gmail.com,Ambos,8.295577e+09,2022-04-06


In [54]:
historico.FBAComp.value_counts()

FBAComp
Ambos      41096
ARS        18791
SEGUROS     1181
AMBAS          4
Name: count, dtype: int64

In [21]:
rango_fechas(historico, 'Creado')

Fecha Inicial:  2014-03-05
Fecha Final:  2022-04-06


In [22]:
historico_group = preprocesamiento(
    df=historico,
    columna_fecha='Creado',
    columna_id_usuario='num_id'
    )

In [23]:
historico_group

,fecha_creacion,num_id
0,2014-03-05,1
1,2014-03-24,1
2,2014-04-17,3
3,2014-04-18,2
4,2014-04-19,1
...,...,...
2671,2022-04-02,19
2672,2022-04-03,22
2673,2022-04-04,52
2674,2022-04-05,70


In [24]:
grafico_fechas(
    df=historico_group,
    columna_fecha='fecha_creacion',
    columna_id_usuario='num_id',
    fecha_inicio='2014-01-01',
    fecha_fin='2014-01-31'
)

In [25]:
grafico_fechas(
    df=full_group,
    columna_fecha='fecha_creacion',
    columna_id_usuario='numero_identificacion',
    fecha_inicio='2014-01-01',
    fecha_fin='2022-01-31'
)

In [26]:
def comparar_dfs(df1, df2, fecha_inicio, fecha_fin):
    
    '''
    df1 = full_extranet
    df2 = historico_app
    '''
    
    # Filtra df1 y df2 por las fechas de inicio y fin
    df1 = df1[(df1['fecha_creacion'] >= fecha_inicio) & (df1['fecha_creacion'] <= fecha_fin)]
    df2 = df2[(df2['fecha_creacion'] >= fecha_inicio) & (df2['fecha_creacion'] <= fecha_fin)]
    
    # Renombra las columnas de valor en cada dataframe
    df1 = df1.rename(columns={'numero_identificacion': 'full_extranet'})
    df2 = df2.rename(columns={'num_id': 'historico_app'})
    
    # Fusiona los dos dataframes en uno solo basándote en la fecha
    df_merged = pd.merge(df1, df2, on='fecha_creacion', how='inner')
    df_merged.set_index('fecha_creacion', inplace=True)
    
    return df_merged

In [27]:
comparar = comparar_dfs(full_group, historico_group, '2022-04-01', '2022-04-06')

In [28]:
comparar

,full_extranet,historico_app
fecha_creacion,,
2022-04-01,45,45
2022-04-02,20,19
2022-04-03,19,22
2022-04-04,48,52
2022-04-05,63,70
2022-04-06,72,2


In [29]:
# Crea el gráfico
fig = go.Figure()
fig.add_trace(go.Scatter(x=comparar.index, y=comparar['full_extranet'], mode='lines+markers', name='full_extranet'))
fig.add_trace(go.Scatter(x=comparar.index, y=comparar['historico_app'], mode='lines+markers', name='historico_app'))
fig.update_layout(autosize = False, width = 1500, height = 600)
fig.show()

Para el ejemplo se usaran los datos de historico_app hasta el 05 de abril de 2022, de esta fecha en adelante se usan los datos de full_extranet

In [30]:
rango_fechas(historico_group, 'fecha_creacion')

Fecha Inicial:  2014-03-05
Fecha Final:  2022-04-06


In [31]:
rango_fechas(full_group, 'fecha_creacion')

Fecha Inicial:  2013-05-15
Fecha Final:  2023-06-05


In [32]:
historico_group_2 = historico_group.copy()

In [33]:
full_group_2 = full_group.copy()

In [34]:
def merge_dataframes(df1, df2, fecha_inicio_df1, fecha_fin_df1, fecha_inicio_df2, fecha_fin_df2):
    # Asegúrate de que las fechas de inicio y fin estén en formato datetime
    fecha_inicio_df1 = pd.to_datetime(fecha_inicio_df1)
    fecha_fin_df1 = pd.to_datetime(fecha_fin_df1)
    fecha_inicio_df2 = pd.to_datetime(fecha_inicio_df2)
    fecha_fin_df2 = pd.to_datetime(fecha_fin_df2)
    
    # Filtra df1 y df2 por las fechas de inicio y fin
    df1 = df1[(df1['fecha_creacion'] >= fecha_inicio_df1) & (df1['fecha_creacion'] <= fecha_fin_df1)]
    df2 = df2[(df2['fecha_creacion'] >= fecha_inicio_df2) & (df2['fecha_creacion'] <= fecha_fin_df2)]
    
    # Renombra las columnas de id en cada dataframe
    df1 = df1.rename(columns={'num_id': 'numero_de_usuarios'})
    df2 = df2.rename(columns={'numero_identificacion': 'numero_de_usuarios'})
    
    # Fusiona los dos dataframes en uno solo basándote en la fecha
    df_merged = pd.concat([df1, df2])
    
    # Reordena el dataframe final por fecha
    df_merged = df_merged.sort_values('fecha_creacion')
    
    return df_merged



In [35]:
# Ejemplo de uso de la función
df_concatenado = merge_dataframes(
    df1=historico_group_2,
    df2=full_group_2,
    fecha_inicio_df1='2014-03-05',
    fecha_fin_df1='2022-04-05',
    fecha_inicio_df2='2022-04-06',
    fecha_fin_df2='2023-06-05'
)

In [36]:
df_concatenado

,fecha_creacion,numero_de_usuarios
0,2014-03-05,1
1,2014-03-24,1
2,2014-04-17,3
3,2014-04-18,2
4,2014-04-19,1
...,...,...
3145,2023-05-30,4
3146,2023-05-31,1
3147,2023-06-01,4
3148,2023-06-02,7


In [37]:
rango_fechas(df_concatenado, 'fecha_creacion')

Fecha Inicial:  2014-03-05
Fecha Final:  2023-06-05


In [38]:
grafico_fechas(
    df=df_concatenado,
    columna_fecha='fecha_creacion',
    columna_id_usuario='numero_de_usuarios',
    fecha_inicio='2019-01-01',
    fecha_fin='2022-12-31'
)

In [39]:
fecha_inicio = '2019-01-01'
fecha_fin = '2022-12-31'
df_modelo = df_concatenado[(df_concatenado['fecha_creacion'] >= fecha_inicio) & (df_concatenado['fecha_creacion'] <= fecha_fin)]


In [40]:
df_modelo

,fecha_creacion,numero_de_usuarios
1491,2019-01-01,1
1492,2019-01-02,12
1493,2019-01-03,19
1494,2019-01-04,14
1495,2019-01-05,3
...,...,...
3075,2022-12-27,47
3076,2022-12-28,38
3077,2022-12-29,43
3078,2022-12-30,26


In [41]:
grafico_fechas(
    df=df_modelo,
    columna_fecha='fecha_creacion',
    columna_id_usuario='numero_de_usuarios',
    fecha_inicio='2019-01-01',
    fecha_fin='2022-12-31'
)

In [42]:
df_modelo.to_parquet('df_modelo.parquet')

In [43]:
def find_missing_dates(df, fecha_col, valor_col, fecha_inicio, fecha_fin):
    '''
    Argumentos:
    
    df: nombre del dataframe
    fecha_col: nombre de la columna con la fechas: 'fecha_col'
    valor_col: nombre de la columna con los valores: 'valor_col
    fecha_inicio: fecha de inicio '2019-01-01'
    fecha_fin: fecha de finalización '2022-12-31'
    '''
    # Fechas de inicio y fin estén en formato datetime
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)

    # Crea un dataframe completo con todas las fechas en el rango
    fechas = pd.date_range(start=fecha_inicio, end=fecha_fin)
    df_completo = pd.DataFrame(fechas, columns=[fecha_col])

    # Fusiona el dataframe completo con el dataframe original
    df_merged = pd.merge(df_completo, df, how='left', on=fecha_col)

    # Encuentra las fechas para las que falta el valor
    fechas_faltantes = df_merged[df_merged[valor_col].isnull()][fecha_col]

    return fechas_faltantes


In [44]:
fechas_faltantes = find_missing_dates(
    df=df_modelo,
    fecha_col='fecha_creacion',
    valor_col='numero_de_usuarios',
    fecha_inicio='2019-01-01',
    fecha_fin='2022-12-31'
)

In [45]:
fechas_faltantes

12    2019-01-13
20    2019-01-21
53    2019-02-23
96    2019-04-07
403   2020-02-08
404   2020-02-09
991   2021-09-18
Name: fecha_creacion, dtype: datetime64[ns]

## Imputar valores faltantes

In [46]:
df_modelo

,fecha_creacion,numero_de_usuarios
1491,2019-01-01,1
1492,2019-01-02,12
1493,2019-01-03,19
1494,2019-01-04,14
1495,2019-01-05,3
...,...,...
3075,2022-12-27,47
3076,2022-12-28,38
3077,2022-12-29,43
3078,2022-12-30,26


In [47]:
df_modelo[df_modelo['fecha_creacion']=='2019-01-13']

,fecha_creacion,numero_de_usuarios


In [48]:
df_modelo['numero_de_usuarios'].mean()

37.70495185694635

In [49]:
def add_missing_dates(df, fecha_col, valor_col, fecha_inicio, fecha_fin):
    '''
    Argumentos:
    
    df: nombre del dataframe
    fecha_col: nombre de la columna con la fechas: 'fecha_col'
    valor_col: nombre de la columna con los valores: 'valor_col
    fecha_inicio: fecha de inicio '2019-01-01'
    fecha_fin: fecha de finalización '2022-12-31'
    '''
    # Fechas de inicio y fin estén en formato datetime
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)

    # Crea un dataframe completo con todas las fechas en el rango
    fechas = pd.date_range(start=fecha_inicio, end=fecha_fin)
    df_completo = pd.DataFrame(fechas, columns=[fecha_col])

    # Calcula el valor medio
    mean_val = df[valor_col].mean()

    # Fusiona el dataframe completo con el dataframe original
    df_merged = pd.merge(df_completo, df, how='left', on=fecha_col)

    # Rellena los valores faltantes con el valor medio
    df_merged[valor_col] = df_merged[valor_col].fillna(mean_val)

    return df_merged


In [50]:
df_modelo_2 = add_missing_dates(
    df=df_modelo,
    fecha_col='fecha_creacion',
    valor_col='numero_de_usuarios',
    fecha_inicio='2019-01-01',
    fecha_fin='2022-12-31'
)

In [51]:
df_modelo_2

,fecha_creacion,numero_de_usuarios
0,2019-01-01,1.0
1,2019-01-02,12.0
2,2019-01-03,19.0
3,2019-01-04,14.0
4,2019-01-05,3.0
...,...,...
1456,2022-12-27,47.0
1457,2022-12-28,38.0
1458,2022-12-29,43.0
1459,2022-12-30,26.0


In [52]:
fechas_faltantes

12    2019-01-13
20    2019-01-21
53    2019-02-23
96    2019-04-07
403   2020-02-08
404   2020-02-09
991   2021-09-18
Name: fecha_creacion, dtype: datetime64[ns]

In [53]:
df_modelo_2[df_modelo_2['fecha_creacion'] == '2019-01-21']

,fecha_creacion,numero_de_usuarios
20,2019-01-21,37.704952
